In [1]:
%reset -f

In [2]:
import pandas as pd
import textdistance
import spacy

In [3]:
# Functions

def add_distance(df, col_name, txt_dist_pkg):
    df\
    .insert(\
            len(df.columns),\
            col_name,\
            df.apply\
            (\
                lambda x: txt_dist_pkg\
                 (\
                    x.subject_label.lower(), x.object_label.lower()\
                    if pd.notnull(x.object_label) else "99"\
                 ),\
                axis=1,\
            )\
           )


In [11]:
df = pd.read_csv("input/unmapped_icd_lex.tsv", sep='\t', low_memory=False, encoding = "ISO-8859-1")
new_df = df[["subject_id", "subject_label", "predicate_id", "object_id", "object_label", "mapping_justification", "confidence", "subject_match_field", "object_match_field", "match_string", "notes"]]
add_distance(new_df, "levenshtein_dist", textdistance.levenshtein.distance)

nlp = spacy.load("en_ner_craft_md")

new_df["semantic_similarity"] = new_df.apply(lambda x: nlp(x.subject_label).similarity(nlp(x.object_label)), axis=1 )
new_df = new_df[["subject_id", "subject_label", "predicate_id", "object_id", "object_label", "levenshtein_dist", "semantic_similarity", "mapping_justification", "confidence", "subject_match_field", "object_match_field", "match_string", "notes"]]
df.head()

<ipython-input-11-8daac3cb0576>:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  new_df["semantic_similarity"] = new_df.apply(lambda x: nlp(x.subject_label).similarity(nlp(x.object_label)), axis=1 )
<ipython-input-11-8daac3cb0576>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["semantic_similarity"] = new_df.apply(lambda x: nlp(x.subject_label).similarity(nlp(x.object_label)), axis=1 )


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string,review,notes,Questions
0,MONDO:0006834,lip cancer,ICD10CM:C00.9,MONDO:equivalentTo,semapv:LexicalMatching,"Malignant neoplasm of lip, unspecified",LEXMATCH,oaklib,0.8,oio:hasExactSynonym,rdfs:label,"malignant neoplasm of lip, unspecified",N,I don't think we want to map to these unspecfi...,NaN
1,MONDO:0006834,lip cancer,ICD10CM:C00.5,MONDO:equivalentTo,semapv:LexicalMatching,"Malignant neoplasm of lip, unspecified, inner ...",LEXMATCH,oaklib,0.8,oio:hasExactSynonym,rdfs:label,"malignant neoplasm of lip, unspecified, inner ...",N,I don't think we want to map to these unspecfi...,NaN
2,MONDO:0002691,liver cancer,ICD10CM:C22.9,MONDO:equivalentTo,semapv:LexicalMatching,"Malignant neoplasm of liver, not specified as ...",LEXMATCH,oaklib,0.8,oio:hasExactSynonym,rdfs:label,"malignant neoplasm of liver, not specified as ...",N,I don't think we want to map to these unspecfi...,NaN
3,MONDO:0001528,vulva cancer,ICD10CM:C51.9,MONDO:equivalentTo,semapv:LexicalMatching,"Malignant neoplasm of vulva, unspecified",LEXMATCH,oaklib,0.8,oio:hasExactSynonym,rdfs:label,"malignant neoplasm of vulva, unspecified",N,I don't think we want to map to these unspecfi...,NaN
4,MONDO:0017775,melioidosis,ICD10CM:A24.9,MONDO:equivalentTo,semapv:LexicalMatching,"Melioidosis, unspecified",LEXMATCH,oaklib,0.8,oio:hasExactSynonym,oio:hasExactSynonym,whitmore's disease,N,I don't think we want to map to these unspecfi...,NaN


In [5]:
df['notes'].value_counts()

other' terms are not in scope for Mondo                    262
MONDO:mondoIsNarrowerThanSource                             70
I don't think we want to map to these unspecfied terms?     12
this is equiv to MONDO:0001356                               3
this is equiv to MONDO:0018531                               3
this is equiv to MONDO:0004948                               2
this is equiv to MONDO:0002903                               2
this is equiv to MONDO:0001442                               2
this is equiv to MONDO:0009692                               2
this is equiv to MONDO:0004696                               2
this is equiv to MONDO:0006861                               2
this is equiv to MONDO:0006000                               2
incorrect match on abbreviation                              1
this is equiv to MONDO:0020311                               1
this is equiv to MONDO:0017598                               1
this is equiv to MONDO:0002354                         

In [6]:
new_df = new_df.sort_values(by=["levenshtein_dist", "semantic_similarity"])
new_df.head()

,subject_id,subject_label,predicate_id,object_id,object_label,levenshtein_dist,semantic_similarity,mapping_justification,confidence,subject_match_field,object_match_field,match_string,notes
395,MONDO:0001694,diffuse interstitial keratitis,ICD10CM:H16.32,MONDO:equivalentTo,Diffuse interstitial keratitis,0,0.958598,semapv:LexicalMatching,0.849779,rdfs:label,rdfs:label,diffuse interstitial keratitis,NaN
381,MONDO:0015358,hereditary motor and sensory neuropathy,ICD10CM:G60.0,MONDO:equivalentTo,Hereditary motor and sensory neuropathy,0,0.975589,semapv:LexicalMatching,0.849779,rdfs:label,rdfs:label,hereditary motor and sensory neuropathy,NaN
357,MONDO:0009756,Niemann-Pick disease type A,ICD10CM:E75.240,MONDO:equivalentTo,Niemann-Pick disease type A,0,1.000000,semapv:LexicalMatching,0.849779,rdfs:label,rdfs:label,niemann-pick disease type a,NaN
358,MONDO:0018982,Niemann-Pick disease type C,ICD10CM:E75.242,MONDO:equivalentTo,Niemann-Pick disease type C,0,1.000000,semapv:LexicalMatching,0.849779,rdfs:label,rdfs:label,niemann-pick disease type c,NaN
444,MONDO:0017078,cephalocele,ICD10CM:Q01,MONDO:equivalentTo,Encephalocele,2,0.410952,semapv:LexicalMatching,0.800000,oio:hasExactSynonym,rdfs:label,encephalocele,NaN


In [7]:
lev_less_10 = new_df[(new_df['notes'].isna()) & (new_df["levenshtein_dist"].values < 10)]
lev_less_10.to_csv("lev_less_10.tsv", sep='\t', index=False)

In [8]:
unspecified_df = df[(~df['notes'].isna()) & (df['notes'].str.contains("I don't think we want to"))]
unspecified_df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string,review,notes,Questions
0,MONDO:0006834,lip cancer,ICD10CM:C00.9,MONDO:equivalentTo,semapv:LexicalMatching,"Malignant neoplasm of lip, unspecified",LEXMATCH,oaklib,0.8,oio:hasExactSynonym,rdfs:label,"malignant neoplasm of lip, unspecified",N,I don't think we want to map to these unspecfi...,NaN
1,MONDO:0006834,lip cancer,ICD10CM:C00.5,MONDO:equivalentTo,semapv:LexicalMatching,"Malignant neoplasm of lip, unspecified, inner ...",LEXMATCH,oaklib,0.8,oio:hasExactSynonym,rdfs:label,"malignant neoplasm of lip, unspecified, inner ...",N,I don't think we want to map to these unspecfi...,NaN
2,MONDO:0002691,liver cancer,ICD10CM:C22.9,MONDO:equivalentTo,semapv:LexicalMatching,"Malignant neoplasm of liver, not specified as ...",LEXMATCH,oaklib,0.8,oio:hasExactSynonym,rdfs:label,"malignant neoplasm of liver, not specified as ...",N,I don't think we want to map to these unspecfi...,NaN
3,MONDO:0001528,vulva cancer,ICD10CM:C51.9,MONDO:equivalentTo,semapv:LexicalMatching,"Malignant neoplasm of vulva, unspecified",LEXMATCH,oaklib,0.8,oio:hasExactSynonym,rdfs:label,"malignant neoplasm of vulva, unspecified",N,I don't think we want to map to these unspecfi...,NaN
4,MONDO:0017775,melioidosis,ICD10CM:A24.9,MONDO:equivalentTo,semapv:LexicalMatching,"Melioidosis, unspecified",LEXMATCH,oaklib,0.8,oio:hasExactSynonym,oio:hasExactSynonym,whitmore's disease,N,I don't think we want to map to these unspecfi...,NaN


In [9]:
narrow_match = new_df[new_df['notes'] == "MONDO:mondoIsNarrowerThanSource"]
narrow_match.to_csv("narrow_match.tsv", sep='\t', index=False)
narrow_match.head()

,subject_id,subject_label,predicate_id,object_id,object_label,levenshtein_dist,semantic_similarity,mapping_justification,confidence,subject_match_field,object_match_field,match_string,notes
20,MONDO:0011565,metabolic syndrome X,ICD10CM:E88.81,MONDO:equivalentTo,Metabolic syndrome,2,0.789930,semapv:LexicalMatching,0.8,oio:hasExactSynonym,oio:hasExactSynonym,dysmetabolic syndrome x,MONDO:mondoIsNarrowerThanSource
37,MONDO:0019669,hypochondrogenesis,ICD10CM:Q77.0,MONDO:equivalentTo,Achondrogenesis,4,0.133580,semapv:LexicalMatching,0.8,rdfs:label,oio:hasExactSynonym,hypochondrogenesis,MONDO:mondoIsNarrowerThanSource
38,MONDO:0007793,hypochondroplasia,ICD10CM:Q77.4,MONDO:equivalentTo,Achondroplasia,4,1.000000,semapv:LexicalMatching,0.8,oio:hasExactSynonym,oio:hasExactSynonym,hypochondroplasia,MONDO:mondoIsNarrowerThanSource
40,MONDO:0001838,acute gonococcal prostatitis,ICD10CM:A54.22,MONDO:equivalentTo,Gonococcal prostatitis,6,0.765738,semapv:RegularExpressionReplacement,0.8,oio:hasExactSynonym,rdfs:label,gonococcal prostatitis,MONDO:mondoIsNarrowerThanSource
364,MONDO:0006040,lactic acidosis,ICD10CM:E87.2,MONDO:equivalentTo,Acidosis,7,0.283362,semapv:LexicalMatching,0.8,rdfs:label,oio:hasExactSynonym,lactic acidosis,MONDO:mondoIsNarrowerThanSource
